# Timetable Generation

This notebook provides a function to generate a timetable from a CSV or Excel file containing course information. The function ensures no time conflicts for courses with the same lecturer or within the same program.

## Required Libraries
- pandas: For data manipulation
- numpy: For numerical operations
- datetime: For handling time-related operations

## Function: `generate_timetable`

### Parameters:
- `file_path`: Path to the input file (CSV or Excel)
- `days`: Number of days to be included in the timetable
- `start_time`: Start time for classes each day (e.g., '9am')
- `end_time`: End time for classes each day (e.g., '5pm')

### Returns:
- A pandas DataFrame representing the structured timetable

### Error Handling:
- Ensures the input file is in a supported format (CSV or Excel)
- Validates the presence of required columns
- Manages time conflicts for lecturers and programs

## Example Usage
The example demonstrates how to use the function with a sample input file and parameters. The generated timetable is displayed in tabular format and saved to a CSV file.


In [28]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import itertools

def generate_timetable(file_path, days, start_time, end_time):
    """
    Generate a timetable from a CSV or Excel file.
    
    Parameters:
    file_path (str): Path to the input file (CSV or Excel)
    days (int): Number of days to be included in the timetable
    start_time (str): Start time for classes each day (e.g., '9am')
    end_time (str): End time for classes each day (e.g., '5pm')
    
    Returns:
    pd.DataFrame: A DataFrame representing the structured timetable
    
    Raises:
    ValueError: If the input file format is unsupported or required columns are missing
    """
    try:
        # Read the file
        if file_path.endswith('.csv'):
            data = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            data = pd.read_excel(file_path)
        else:
            raise ValueError("Unsupported file format. Please use CSV or Excel file.")
        
        # Check if required columns are present
        required_columns = ['course name', 'course unit', 'lecture name', 'program']
        if not all(column in data.columns for column in required_columns):
            raise ValueError(f"File must contain the following columns: {', '.join(required_columns)}")
        
        # Convert start_time and end_time to datetime objects
        start_time = datetime.strptime(start_time, '%I%p')
        end_time = datetime.strptime(end_time, '%I%p')
        time_slots = []
        
        # Generate time slots
        current_time = start_time
        while current_time < end_time:
            time_slots.append(current_time.strftime('%I%p'))
            current_time += timedelta(minutes=60)  # assuming each slot is 1 hour
        time_slots.append(end_time.strftime('%I%p'))

        # Initialize timetable dictionary
        timetable = {time: {day: [] for day in range(1, days + 1)} for time in time_slots}

        # Schedule courses
        for _, row in data.iterrows():
            scheduled = False
            for time in time_slots:
                for day in range(1, days + 1):
                    if len(timetable[time][day]) == 0 or all(course['program'] != row['program'] and course['lecture name'] != row['lecture name'] for course in timetable[time][day]):
                        timetable[time][day].append({'course name': row['course name'], 'program': row['program'], 'lecture name': row['lecture name']})
                        scheduled = True
                        break
                if scheduled:
                    break
            if not scheduled:
                raise ValueError(f"Could not schedule course {row['course name']} without conflicts.")
        
        # Convert timetable to DataFrame for display
        timetable_df = pd.DataFrame(index=range(1, days + 1), columns=time_slots)
        for time in timetable:
            for day in timetable[time]:
                courses = ', '.join([course['course name'] + ' (' + course['program'] + ')' for course in timetable[time][day]])
                timetable_df.at[day, time] = courses

        return timetable_df
    except Exception as e:
        print(f"Error: {e}")




Enter the path to the CSV or Excel file:  student_timetable_sample.csv
Enter the number of days to be included in the timetable:  5
Enter the start time for classes each day (e.g., '9am'):  2pm
Enter the end time for classes each day (e.g., '5pm'):  6pm


                                                02PM 03PM 04PM 05PM 06PM
1  Mathematics I (Computer Science), Physics I (M...                    
2  Mathematics I (Computer Science), Physics I (M...                    
3  Mathematics I (Computer Science), Physics I (M...                    
4  Mathematics I (Computer Science), Physics I (M...                    
5  Mathematics I (Computer Science), Physics I (M...                    


In [4]:
# Prompt user for input
file_path = input("Enter the path to the CSV or Excel file: ")
days = int(input("Enter the number of days to be included in the timetable: "))
start_time = input("Enter the start time for classes each day (e.g., '9am'): ")
end_time = input("Enter the end time for classes each day (e.g., '5pm'): ")

# Generate and display timetable
timetable_df = generate_timetable(file_path, days, start_time, end_time)
print(timetable_df)


Enter the path to the CSV or Excel file:  'student_timetable_sample.csv'
Enter the number of days to be included in the timetable:  5
Enter the start time for classes each day (e.g., '9am'):  '9am'
Enter the end time for classes each day (e.g., '5pm'):  '2pm'


Error: Unsupported file format. Please use CSV or Excel file.
None


In [25]:
# Prompt user for input
file_path = input("Enter the path to the CSV or Excel file: ").strip()
days = int(input("Enter the number of days to be included in the timetable: "))
start_time = input("Enter the start time for classes each day (e.g., '9am'): ").strip()
end_time = input("Enter the end time for classes each day (e.g., '5pm'): ").strip()

Enter the path to the CSV or Excel file:  student_timetable_sample.csv
Enter the number of days to be included in the timetable:  5
Enter the start time for classes each day (e.g., '9am'):  9am
Enter the end time for classes each day (e.g., '5pm'):  2pm


In [27]:
# Generate and display timetable
timetable_df = generate_timetable(file_path, days, start_time, end_time)
timetable_df

,1,2,3,4,5
09AM,"Mathematics I (Computer Science), Physics I (M...",,,,
10AM,"Mathematics I (Computer Science), Physics I (M...",,,,
11AM,"Mathematics I (Computer Science), Physics I (M...",,,,
12PM,"Mathematics I (Computer Science), Physics I (M...",,,,
01PM,"Mathematics I (Computer Science), Physics I (M...",,,,
02PM,,,,,


In [22]:
# Save the timetable to a CSV file
timetable_df.to_csv('timetable.csv')